In [1]:
import json
import pandas as pd
from pathlib import Path

In [2]:
root = Path.home() / "Downloads/mmlu_output/"

choices = ('A', 'B', 'C', 'D')
files_list = []
predictions_list = []
correct_list = []
for f in root.glob('*/*/attentions/*_attentions.pt'):
    if f.is_file() and f.stat().st_size > 2e6:  # Exclude the empty files
        files_list.append(f.relative_to(root))
        q_id = str(f.stem).split('_')[0]
        ans_file = next(iter(f.parent.parent.glob("*.json")))
        with open(ans_file, "r") as infile:
            ans_json = json.load(infile)
            full_ans = ans_json[q_id]["Full Answer"]
            pred = ans_json[q_id]["Predicted Answer"]
            correct = ans_json[q_id]["Correct Answer"]
            if pred not in choices and full_ans[0] in choices:
                pred = full_ans[0]
        predictions_list.append(pred)
        correct_list.append(correct)

df = pd.DataFrame()
df['filename'] = files_list
df['prediction'] = predictions_list
df['correct'] = correct_list

In [3]:
df['correctness'] = df['prediction'] == df['correct']
df['dataset'] = df['filename'].apply(lambda filename: str(Path(filename).parent.parent.name))
df

,filename,prediction,correct,correctness,dataset
0,auxiliary_train\arc_hard\attentions\0_attentio...,B,A,False,arc_hard
1,auxiliary_train\arc_hard\attentions\1000_atten...,A,A,True,arc_hard
2,auxiliary_train\arc_hard\attentions\1001_atten...,A,B,False,arc_hard
3,auxiliary_train\arc_hard\attentions\1002_atten...,A,A,True,arc_hard
4,auxiliary_train\arc_hard\attentions\1003_atten...,A,A,True,arc_hard
...,...,...,...,...,...
1730,auxiliary_train\science_elementary\attentions\...,B,B,True,science_elementary
1731,auxiliary_train\science_elementary\attentions\...,C,C,True,science_elementary
1732,auxiliary_train\science_elementary\attentions\...,C,C,True,science_elementary
1733,auxiliary_train\science_elementary\attentions\...,B,B,True,science_elementary


In [4]:
# Accuracy overall
df['correctness'].mean()

np.float64(0.7285302593659942)

In [5]:
# Accuracy per dataset
df.groupby('dataset')['correctness'].mean()

dataset
arc_hard              0.679785
science_elementary    0.816856
Name: correctness, dtype: float64

In [6]:
df['dataset'].value_counts()

dataset
arc_hard              1118
science_elementary     617
Name: count, dtype: int64